In [1]:
import pandas as pd
# Geopandas is a geo dataframe
# import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

# Load combined data set

In [2]:
all = pd.read_csv("/home/kim/data/bedmap/bedmap123.csv")
# loading data takes 2 min, 67.7 M data points

KeyboardInterrupt: 

In [3]:
all.head()

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
0,246,-162.2167,-78.17500,-393747.895145,-1.227610e+06,43.0,257.0,-607.0,False,BEDMAP1_1966-2000_AIR_BM1.csv
1,247,-163.0000,-78.78833,-357254.891928,-1.168528e+06,197.0,415.0,-218.0,True,BEDMAP1_1966-2000_AIR_BM1.csv
2,248,-164.8333,-78.91666,-316007.544874,-1.165778e+06,48.0,369.0,-425.0,False,BEDMAP1_1966-2000_AIR_BM1.csv
3,249,-168.6500,-79.15334,-232601.272638,-1.158792e+06,59.0,384.0,-539.0,False,BEDMAP1_1966-2000_AIR_BM1.csv
4,250,-171.5167,-79.11667,-174949.002700,-1.172950e+06,49.0,328.0,-695.0,False,BEDMAP1_1966-2000_AIR_BM1.csv


In [4]:
all.shape

(67740122, 10)

# Preprocess

## Check bounds

In [5]:
all = all.drop(all.loc[(all['x'] > 3333000) | (all['x'] < -3333000) | (all['y'] > 3333000) | (all['y'] < -3333000)].index)

all.shape

## Check b,s,t, replaced with 9999 instead of -9999

### b == 9999

In [6]:
# 6 values from BEDMAP1_1966-2000_AIR_BM1.csv but with s > 10000 so I am removing these
all.loc[all["b"] == 9999]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
18564,18810,-164.7083,-79.31028,-307172.524588,-1.123473e+06,10344.0,345.0,9999.0,False,BEDMAP1_1966-2000_AIR_BM1.csv
18565,18811,-167.2167,-79.23333,-259574.040108,-1.144063e+06,10411.0,412.0,9999.0,False,BEDMAP1_1966-2000_AIR_BM1.csv
18567,18813,-171.8200,-79.37167,-164761.782062,-1.146202e+06,10356.0,357.0,9999.0,False,BEDMAP1_1966-2000_AIR_BM1.csv
18581,18827,165.0000,-78.96667,311195.425496,-1.161397e+06,10508.0,509.0,9999.0,False,BEDMAP1_1966-2000_AIR_BM1.csv
18592,18838,170.8908,-77.32528,218881.565372,-1.365120e+06,10160.0,161.0,9999.0,False,BEDMAP1_1966-2000_AIR_BM1.csv


In [7]:
all = all.drop(all.loc[all["b"] == 9999].index)

### s == 9999

In [8]:
# 2 values but also something clearly wrong
all.loc[all["s"] == 9999]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
18631439,138192,-57.517011,-83.048743,-6.378606e+05,406095.844400,9999.0,205.21,10204.21,False,NASA_2010_ICEBRIDGE_AIR_BM2.csv
18843942,350695,-69.999498,-74.920739,-1.548175e+06,563505.160573,9999.0,687.85,10686.86,False,NASA_2010_ICEBRIDGE_AIR_BM2.csv


In [9]:
all = all.drop(all.loc[all["s"] == 9999].index)

### t == 9999

In [10]:
all.loc[all["t"] == 9999]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source


### t, b given, s missing

In [12]:
# Surface missing, t and b given: can't check really
all.loc[(all['s'] == -9999) & (all['t'] != -9999) & (all['b'] != -9999)]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
27284703,196992,124.733357,-75.825621,1.271892e+06,-881795.399322,-9999.0,4384.13,-1249.160,False,UTIG_2016_OLDICE_AIR_BM3.csv
27292023,205919,123.372746,-75.845684,1.290624e+06,-850128.934768,-9999.0,3509.28,-359.870,False,UTIG_2016_OLDICE_AIR_BM3.csv
27296095,248070,121.026688,-75.306079,1.375355e+06,-827269.092831,-9999.0,3619.47,-447.900,False,UTIG_2016_OLDICE_AIR_BM3.csv
27296096,248336,121.214997,-75.290907,1.374061e+06,-832652.849269,-9999.0,3734.10,-559.120,False,UTIG_2016_OLDICE_AIR_BM3.csv
27301896,254280,122.757047,-75.080422,1.370702e+06,-881903.198078,-9999.0,3343.22,-144.260,False,UTIG_2016_OLDICE_AIR_BM3.csv
...,...,...,...,...,...,...,...,...,...,...
40726408,564207,74.083000,-72.269000,1.866985e+06,532423.493472,-9999.0,1717.04,-172.009,False,PRIC_2018_CHA4_AIR_BM3.csv
40726409,564208,74.082000,-72.269000,1.866975e+06,532456.078419,-9999.0,1718.73,-173.803,False,PRIC_2018_CHA4_AIR_BM3.csv
40726410,564209,74.082000,-72.269000,1.866975e+06,532456.078419,-9999.0,1711.97,-167.147,False,PRIC_2018_CHA4_AIR_BM3.csv
40726411,564210,74.082000,-72.269000,1.866975e+06,532456.078419,-9999.0,1710.28,-165.561,False,PRIC_2018_CHA4_AIR_BM3.csv


## b, s given, t missing

In [13]:
# Thickness missing, b & s given
# Check if b > s: 92 k
# Bedmap paper states imputation
bed_surface_threshold = 0.0
all.loc[(all['s'] != -9999) & (all['t'] == -9999) & (all['b'] != -9999) & (all['b'] - all['s'] > bed_surface_threshold)]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
2843597,5733,162.644464,-82.838027,2.324175e+05,-7.436671e+05,983.081801,-9999.0,1041.798601,False,UTIG_2000_Robb-Glacier_AIR_BM2.csv
2861295,23541,163.171671,-82.796760,2.268679e+05,-7.500821e+05,1165.329928,-9999.0,1189.999628,False,UTIG_2000_Robb-Glacier_AIR_BM2.csv
3959403,20647,161.764000,-77.624000,4.223715e+05,-1.281935e+06,1561.400000,-9999.0,1562.800000,False,UTIG_1999_SOAR-LVS-WLK_AIR_BM2.csv
3959404,20648,161.712000,-77.625000,4.235003e+05,-1.281447e+06,1661.500000,-9999.0,1661.900000,False,UTIG_1999_SOAR-LVS-WLK_AIR_BM2.csv
3959405,20649,161.673000,-77.625000,4.243724e+05,-1.281158e+06,1572.100000,-9999.0,1575.500000,False,UTIG_1999_SOAR-LVS-WLK_AIR_BM2.csv
...,...,...,...,...,...,...,...,...,...,...
67626376,9177939,100.754946,-66.330602,2.561667e+06,-4.865768e+05,75.080000,-9999.0,75.090000,False,UTIG_2010_ICECAP_AIR_BM3.csv
67626379,9177942,100.755979,-66.330245,2.561698e+06,-4.866306e+05,80.220000,-9999.0,80.270000,False,UTIG_2010_ICECAP_AIR_BM3.csv
67626380,9177943,100.756320,-66.330126,2.561709e+06,-4.866483e+05,77.620000,-9999.0,77.790000,False,UTIG_2010_ICECAP_AIR_BM3.csv
67626381,9177944,100.756669,-66.330006,2.561719e+06,-4.866665e+05,78.240000,-9999.0,78.820000,False,UTIG_2010_ICECAP_AIR_BM3.csv


In [14]:
# remove
all = all.drop(all.loc[(all['s'] != -9999) & (all['t'] == -9999) & (all['b'] != -9999) & (all['b'] - all['s'] > bed_surface_threshold)].index)

### traingle:

In [15]:
# Inspect values where all 3 fields are given and b > s
# > 2 M 
all.loc[(all['s'] != -9999) & (all['t'] != -9999) & (all['b'] != -9999) & (all['s'] < all['b'])]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
17412,17658,-98.866670,-77.506670,-1.346334e+06,-2.100279e+05,1444.00,3399.0,1955.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
18977,19223,-77.400000,-75.150000,-1.583149e+06,3.538757e+05,469.00,2090.0,1620.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
71965,73253,-57.745590,-75.561310,-1.333495e+06,8.415160e+05,-42.00,345.0,0.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
71966,73254,-57.775890,-75.556380,-1.334400e+06,8.411007e+05,-11.00,731.0,0.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
71967,73255,-57.823610,-75.553150,-1.335401e+06,8.401788e+05,-8.00,249.0,0.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
...,...,...,...,...,...,...,...,...,...,...
67574938,9077155,167.344145,-78.054864,2.853486e+05,-1.270756e+06,-41.43,-0.0,-41.42,False,UTIG_2010_ICECAP_AIR_BM3.csv
67614166,9153672,166.219097,-77.972720,3.123929e+05,-1.273668e+06,-46.87,-0.0,-46.86,False,UTIG_2010_ICECAP_AIR_BM3.csv
67614185,9153691,166.236773,-77.972770,3.119986e+05,-1.273759e+06,-46.97,-0.0,-46.96,False,UTIG_2010_ICECAP_AIR_BM3.csv
67614268,9153792,166.330773,-77.973129,3.098991e+05,-1.274231e+06,-45.07,-0.0,-45.06,False,UTIG_2010_ICECAP_AIR_BM3.csv


In [16]:
# deleting an extra 100k because of inconsistency
traingle_threshold = 5.0
all.loc[(all['s'] != -9999) & (all['t'] != -9999) & (all['b'] != -9999) & (np.abs(all['s'] - all['t'] - all['b']) > traingle_threshold)]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
0,246,-162.216700,-78.175000,-393747.895145,-1.227610e+06,43.00,257.00,-607.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
2,248,-164.833300,-78.916660,-316007.544874,-1.165778e+06,48.00,369.00,-425.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
3,249,-168.650000,-79.153340,-232601.272638,-1.158792e+06,59.00,384.00,-539.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
4,250,-171.516700,-79.116670,-174949.002700,-1.172950e+06,49.00,328.00,-695.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
5,251,-173.983300,-78.995000,-125704.556827,-1.192655e+06,51.00,344.00,-536.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
...,...,...,...,...,...,...,...,...,...,...
67664053,9242348,170.283944,-77.507922,229939.718563,-1.342937e+06,2075.80,1318.60,-1318.60,False,UTIG_2010_ICECAP_AIR_BM3.csv
67664054,9242349,170.283193,-77.508000,229955.874155,-1.342925e+06,2075.54,1318.46,-1318.46,False,UTIG_2010_ICECAP_AIR_BM3.csv
67670200,9251585,171.570298,-78.030899,191312.107207,-1.290932e+06,3279.80,2164.64,-2164.64,False,UTIG_2010_ICECAP_AIR_BM3.csv
67675397,9259096,170.051281,-77.483218,235860.192867,-1.344666e+06,-28.94,101.58,-101.58,False,UTIG_2010_ICECAP_AIR_BM3.csv


In [17]:
# Triangle check for all complete values
traingle_threshold = 5.0
all = all.drop(all.loc[(all['s'] != -9999) & (all['t'] != -9999) & (all['b'] != -9999) & (np.abs(all['s'] - all['t'] - all['b']) > traingle_threshold)].index)

In [18]:
all.shape

(65303498, 10)

In [19]:
# Repeat with stricter threshold for b > s
# bed elevation > surface
bed_surface_threshold = 0.0
all = all.drop(all.loc[(all['s'] != -9999) & (all['t'] != -9999) & (all['b'] != -9999) & (all['b'] - all['s'] > bed_surface_threshold)].index)

# Very high bedrock

In [20]:
all.shape

(65299312, 10)

In [21]:
all.loc[(all['b'] > 3600)]["source"].value_counts()

NASA_2010_ICEBRIDGE_AIR_BM2.csv               61295
UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv       11
BEDMAP1_1966-2000_AIR_BM1.csv                     1
NASA_2012_ICEBRIDGE_AIR_BM2.csv                   1
Name: source, dtype: int64

In [22]:
all.loc[(all['b'] > 3600) & (all["source"] == "BEDMAP1_1966-2000_AIR_BM1.csv")]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
57338,58626,-58.56667,-78.13499,-1.103764e+06,674621.76024,9356.0,154.0,9202.0,False,BEDMAP1_1966-2000_AIR_BM1.csv


In [23]:
# appears to be an error
all.loc[(all['b'] > 3600) & (all["source"] == "NASA_2012_ICEBRIDGE_AIR_BM2.csv")]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
3385894,561425,-77.963492,-71.667815,-1.964165e+06,418804.429005,3642.84,0.0,3642.84,False,NASA_2012_ICEBRIDGE_AIR_BM2.csv


In [24]:
# way too high
all.loc[(all['b'] > 3600) & (all["source"] == "UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv")]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
4601260,223897,-155.35427,-77.921821,-549200.980741,-1.197034e+06,6456.9,331.9,6125.0,False,UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv
4601518,224159,-155.24392,-77.896944,-552649.473847,-1.198455e+06,7029.1,92.3,6936.8,False,UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv
4601666,224307,-155.18583,-77.882902,-554511.472307,-1.199294e+06,6614.7,360.7,6254.0,False,UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv
4602011,224652,-155.04342,-77.849388,-559043.700306,-1.201249e+06,6506.3,754.8,5751.5,False,UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv
4602246,224887,-154.95268,-77.828061,-561937.154912,-1.202484e+06,6674.1,700.1,5974.0,False,UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv
4602489,225130,-154.83719,-77.800531,-565645.580823,-1.204086e+06,6851.7,1015.2,5836.5,False,UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv
4602783,225424,-154.71267,-77.770533,-569668.623356,-1.205833e+06,7769.0,456.0,7313.0,False,UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv
4603185,225826,-154.56485,-77.734574,-574474.277781,-1.207927e+06,8942.9,686.0,8256.9,False,UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv
4605304,227945,-153.76426,-77.535809,-600950.442926,-1.219372e+06,5411.8,235.7,5176.1,False,UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv
4605432,228073,-153.71485,-77.523557,-602598.031692,-1.220060e+06,5471.0,339.3,5131.7,False,UTIG_1998_West-Marie-Byrd-Land_AIR_BM2.csv


In [25]:
# way too high
all.loc[(all['b'] > 3600) & (all["source"] == "NASA_2010_ICEBRIDGE_AIR_BM2.csv")]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
18506498,13251,-57.981771,-64.528703,-2.384386e+06,1.490985e+06,3600.99,0.0,3600.99,False,NASA_2010_ICEBRIDGE_AIR_BM2.csv
18506499,13252,-57.981970,-64.528621,-2.384399e+06,1.490981e+06,3608.58,0.0,3608.58,False,NASA_2010_ICEBRIDGE_AIR_BM2.csv
18506500,13253,-57.982169,-64.528539,-2.384412e+06,1.490978e+06,3616.17,0.0,3616.17,False,NASA_2010_ICEBRIDGE_AIR_BM2.csv
18506501,13254,-57.982369,-64.528457,-2.384425e+06,1.490975e+06,3623.77,0.0,3623.77,False,NASA_2010_ICEBRIDGE_AIR_BM2.csv
18506502,13255,-57.982568,-64.528376,-2.384438e+06,1.490971e+06,3631.36,0.0,3631.36,False,NASA_2010_ICEBRIDGE_AIR_BM2.csv
...,...,...,...,...,...,...,...,...,...,...
19482627,989380,-61.849948,-63.963643,-2.536389e+06,1.357154e+06,3601.83,0.0,3601.83,False,NASA_2010_ICEBRIDGE_AIR_BM2.csv
19482628,989381,-61.849773,-63.963749,-2.536374e+06,1.357156e+06,3601.43,0.0,3601.43,False,NASA_2010_ICEBRIDGE_AIR_BM2.csv
19482629,989382,-61.849597,-63.963855,-2.536359e+06,1.357159e+06,3601.03,0.0,3601.03,False,NASA_2010_ICEBRIDGE_AIR_BM2.csv
19482630,989383,-61.849422,-63.963961,-2.536345e+06,1.357161e+06,3600.63,0.0,3600.63,False,NASA_2010_ICEBRIDGE_AIR_BM2.csv


In [26]:
# delete
all = all.drop(all.loc[(all['b'] > 3600)].index)

### Very high surface

In [27]:
# plausible
all.loc[(all['s'] > 4200)]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
92372,93660,95.904300,-76.974400,1.413605e+06,-1.461891e+05,4278.0000,2413.00,1865.0000,False,BEDMAP1_1966-2000_AIR_BM1.csv
92373,93661,95.811000,-76.957790,1.415659e+06,-1.440720e+05,4205.0000,2414.00,1791.0000,False,BEDMAP1_1966-2000_AIR_BM1.csv
701822,1273541,158.710200,-75.577600,5.718672e+05,-1.467536e+06,4863.0000,4470.00,393.0000,False,BEDMAP1_1966-2000_AIR_BM1.csv
784216,1480422,95.998790,-76.992290,1.411405e+06,-1.483145e+05,4244.0000,2575.00,1669.0000,False,BEDMAP1_1966-2000_AIR_BM1.csv
37068436,660730,99.851394,-77.679050,1.323863e+06,-2.298939e+05,4292.6073,3955.41,337.1973,True,PRIC_2015_CHA1_AIR_BM3.csv
37068437,660731,99.852117,-77.679173,1.323847e+06,-2.299083e+05,4292.5695,3955.41,337.1595,True,PRIC_2015_CHA1_AIR_BM3.csv
37068438,660732,99.852839,-77.679296,1.323830e+06,-2.299227e+05,4292.5316,3955.41,337.1216,True,PRIC_2015_CHA1_AIR_BM3.csv
37068439,660733,99.853563,-77.679419,1.323814e+06,-2.299371e+05,4292.4937,3955.41,337.0837,True,PRIC_2015_CHA1_AIR_BM3.csv
37068440,660734,99.854284,-77.679543,1.323798e+06,-2.299514e+05,4292.4757,3955.41,337.0657,True,PRIC_2015_CHA1_AIR_BM3.csv
37068441,660735,99.855005,-77.679668,1.323781e+06,-2.299657e+05,4292.4838,3955.41,337.0738,True,PRIC_2015_CHA1_AIR_BM3.csv


In [28]:
# ice is thick where the bed is very low
all.loc[(all['t'] > 4700)]

,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
93608,94896,135.202500,-69.938000,1.551136e+06,-1.562140e+06,2435.00,4776.00,-2341.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
94017,95305,118.105500,-76.075700,1.340851e+06,-7.161132e+05,3246.00,4716.00,-1470.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
784462,1480668,135.135600,-69.972600,1.550228e+06,-1.557583e+06,2535.00,4730.00,-2195.00,False,BEDMAP1_1966-2000_AIR_BM1.csv
34391951,286518,118.383223,-76.095101,1.335483e+06,-7.215877e+05,3212.45,4719.58,-1507.13,False,NASA_2013_ICEBRIDGE_AIR_BM3.csv
34391952,286519,118.382272,-76.095242,1.335481e+06,-7.215582e+05,3212.58,4729.43,-1516.85,False,NASA_2013_ICEBRIDGE_AIR_BM3.csv
...,...,...,...,...,...,...,...,...,...,...
67142901,7993991,118.364609,-76.081390,1.337047e+06,-7.218717e+05,3209.80,4711.05,-1501.25,False,UTIG_2010_ICECAP_AIR_BM3.csv
67142902,7993992,118.365380,-76.081273,1.337049e+06,-7.218958e+05,3209.74,4711.47,-1501.73,False,UTIG_2010_ICECAP_AIR_BM3.csv
67142903,7993993,118.366152,-76.081155,1.337050e+06,-7.219200e+05,3210.06,4712.06,-1502.00,False,UTIG_2010_ICECAP_AIR_BM3.csv
67142904,7993994,118.366923,-76.081037,1.337052e+06,-7.219442e+05,3210.33,4710.24,-1499.91,False,UTIG_2010_ICECAP_AIR_BM3.csv


# Export cleaned data

In [29]:
all.to_csv("/home/kim/data/bedmap/bedmap123_preprocessed.csv")
# takes a 9 minutes